In [ ]:
# import packages

% matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost
from scipy import stats
from statsmodels.stats import stattools
import shap

In [ ]:
# import data

df = pd.read_csv('historical-data.csv')
df_2018 = pd.read_csv('2018-19-data.csv')

# Explore data

In [ ]:
# density plot of historical salary

plt.style.use('fivethirtyeight')

salary_dens, ax = plt.subplots()

ax = sns.kdeplot(df['percent_of_cap'], ax = ax, shade = True, legend = False)

salary_dens.suptitle("Distribution of salary", weight = 'bold', size = 18)

ax.set_xlabel("% of cap")
ax.set_ylabel("Density")

salary_dens.text(x = -0.02, y = -0.08,
    s = '___________________________________________________________',
    fontsize = 14, color = 'grey', horizontalalignment='left', alpha = .3)

salary_dens.text(x = -0.02, y = -.14,
    s = 'https://dribbleanalytics.blog                     ',
    fontsize = 14, fontname = 'Rockwell', color = 'grey', horizontalalignment='left')

salary_dens.savefig('salary_dens.png', dpi = 400, bbox_inches = 'tight')

In [ ]:
# function to produce a scatter plot of a given statistic and salary

def plot_salary_corr(stat):
    
    plt.style.use('fivethirtyeight')
    fig, ax = plt.subplots()
    
    x = df[stat]
    y = df['percent_of_cap']
    
    ax.scatter(x, y, alpha = .25)
    ax.plot(np.unique(x), np.poly1d(np.polyfit(x, y, 1))(np.unique(x)), color = 'C1')

    ax.set_xlabel("%s" % stat.upper())
    ax.set_ylabel("% of yearly cap")

    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

    ax.set_title("r = %s" %str(round(r_value, 3)), size = 14, fontname = 'Rockwell')
    fig.suptitle('%s vs. salary' %stat.upper(), size = 18, weight = 'bold', y = 1.005)

    fig.text(x = -0.03, y = -0.07,
        s = '_____________________________________________________________',
        fontsize = 14, color = 'grey', horizontalalignment='left', alpha = .3)

    fig.text(x = -0.03, y = -.13,
        s = 'https://dribbleanalytics.blog                     ',
        fontsize = 14, fontname = 'Rockwell', color = 'grey', horizontalalignment='left')

    fig.savefig('%s_salary.png' % stat, dpi = 400, bbox_inches = 'tight')

In [ ]:
plot_salary_corr('pts')

In [ ]:
plot_salary_corr('ws')

In [ ]:
plot_salary_corr('age')

# Create models

In [ ]:
# define features and output, split data into 25% test size

features = ['age', 'pts', 'trb', 'ast', 'stl', 'blk', 'ts', 'ws']
output = ['percent_of_cap']

train, test = train_test_split(df, test_size = 0.25, random_state = 0)

xtrain = train[features]
ytrain = train[output]

xtest = test[features]
ytest = test[output]

print("Training size: " + str(len(train)))
print("Testing size: " + str(len(test)))

In [ ]:
# correlation plot of features

corr = df[features].corr()

plt.style.use('fivethirtyeight')
corr_plot, ax = plt.subplots()

cmap = plt.get_cmap('plasma')

ax = sns.heatmap(corr, center=0, cmap = cmap,
            square=True, linewidths=.5)

ax.yaxis.set_tick_params(rotation=0)

corr_plot.suptitle("Feature correlations", weight = 'bold', size = 18)

corr_plot.text(x = 0.17, y = 0,
    s = '_____________________________________________',
    fontsize = 14, color = 'grey', horizontalalignment='left', alpha = .3)

corr_plot.text(x = 0.17, y = -.06,
    s = 'https://dribbleanalytics.blog                     ',
    fontsize = 14, fontname = 'Rockwell', color = 'grey', horizontalalignment='left')

corr_plot.savefig('corr-plot.png', dpi = 400, bbox_inches = 'tight')

In [ ]:
# create grid search function to return best parameters for each model

cv = KFold(n_splits = 3, random_state = 0)

def grid_search(model, grid):
    clf = GridSearchCV(model, grid, cv = cv, n_jobs = -1, verbose = 2, iid = False, scoring = 'neg_mean_squared_error')
    scores(clf)
    
    print(clf.best_params_)

In [ ]:
# function to train and evaluate models

def scores(model):
    
    model.fit(xtrain, ytrain.values.ravel())
    y_pred = model.predict(xtest)
    
    print("MSE: %.3f" % mean_squared_error(ytest, y_pred))
    print('R2 score: %.3f' % r2_score(ytest, y_pred))

    cv_score = cross_val_score(model, xtest, ytest.values.ravel(), cv = 3, scoring = 'neg_mean_squared_error')
    print("MSE cross validation score: %0.3f (+/- %0.3f)" % (cv_score.mean(), cv_score.std() * 2))
    
    cv_score = cross_val_score(model, xtest, ytest.values.ravel(), cv = 3, scoring = 'r2')
    print("R2 cross validation score: %0.3f (+/- %0.3f)" % (cv_score.mean(), cv_score.std() * 2))
    
    return y_pred

In [ ]:
knn = KNeighborsRegressor()

y_knn = scores(knn)

In [ ]:
n_neighbors = [x for x in np.arange(5, 21)]
weights = ['uniform', 'distance']

grid = {'n_neighbors': n_neighbors,
        'weights': weights}

grid_search(knn, grid)

In [ ]:
knn = KNeighborsRegressor(n_neighbors = 19, weights = 'uniform')

y_knn = scores(knn)

In [ ]:
rf = RandomForestRegressor(n_estimators = 100, random_state = 0)

y_rf = scores(rf)

In [ ]:
max_depth = [int(x) for x in np.linspace(start = 10, stop = 100, num = 10)]
max_features = ['auto', 'sqrt']
n_estimators = [int(x) for x in np.linspace(start = 25, stop = 250, num = 10)]
random_state = [0]

grid = {'max_depth': max_depth,
        'max_features': max_features,
        'n_estimators': n_estimators,
        'random_state': random_state}

grid_search(rf, grid)

In [ ]:
rf = RandomForestRegressor(max_depth = 10, max_features = 'auto', n_estimators = 250, random_state = 0)

y_rf = scores(rf)

In [ ]:
gbr = GradientBoostingRegressor(random_state = 0)

y_gbr = scores(gbr)

In [ ]:
loss = ['ls', 'lad', 'huber']
max_depth = [int(x) for x in np.linspace(start = 10, stop = 100, num = 10)]
max_features = ['auto', 'sqrt']
n_estimators = [int(x) for x in np.linspace(start = 25, stop = 250, num = 10)]
random_state = [0]

grid = {'loss': loss,
        'max_depth': max_depth,
        'max_features': max_features,
        'n_estimators': n_estimators,
        'random_state': random_state}

grid_search(gbr, grid)

In [ ]:
gbr = GradientBoostingRegressor(random_state = 0)

y_gbr = scores(gbr)

In [ ]:
xgb = xgboost.XGBRegressor(objective = "reg:squarederror", random_state = 0)

y_xgb = scores(xgb)

In [ ]:
max_depth = [int(x) for x in np.linspace(start = 10, stop = 100, num = 10)]
learning_rate = [0.01, 0.05, 0.1, 0.2]
n_estimators = [int(x) for x in np.linspace(start = 25, stop = 250, num = 10)]
booster = ['gbtree', 'gblinear', 'dart']

grid = {'max_depth': max_depth,
        'learning_rate': learning_rate,
        'n_estimators': n_estimators,
        'booster': booster}

grid_search(xgb, grid)

In [ ]:
xgb = xgboost.XGBRegressor(objective = "reg:squarederror", random_state = 0)

y_xgb = scores(xgb)

# Standardized residuals

In [ ]:
# function to convert model y_pred values into standardized residuals

def residuals(y_pred):
    
    resid = [i for i in (ytest.values.reshape(ytest.shape[0],) - y_pred)]
    ssr = [i ** 2 for i in resid]
    
    ssr_sum = sum(ssr)
        
    stand_resid = []
    for i in resid:
        stand_resid.append(i / ((ssr_sum / (ytest.shape[0] - 2)) ** (1/2)))
        
    return stand_resid

In [ ]:
knn_resid = residuals(y_knn)
rf_resid = residuals(y_rf)
gbr_resid = residuals(y_gbr)
xgb_resid = residuals(y_xgb)

In [ ]:
# function to find outliers in standardized residuals (points more than 2 stdev away from mean)

def outliers(x):
    
    np_list = np.array(x)
    stdev = np.std(np_list)
    mean = np.mean(np_list)

    outliers = 0
    for i in x:
        if i < mean - 2 * stdev:
            outliers += 1
        elif i > mean + 2 * stdev:
            outliers += 1

    outlier_percent = 1 - outliers / ytest.shape[0]
    outlier_string = "{:.3%}".format(outlier_percent)
    
    return outlier_string

In [ ]:
# plot standardized residuals

plt.style.use('fivethirtyeight')

resid_fig, ax = plt.subplots(2, 2, sharex = True, sharey = True)

norm = np.random.standard_normal(10000)

ax1 = sns.kdeplot(knn_resid, ax=ax[0, 0])
ax1 = sns.kdeplot(norm, ax=ax[0, 0])
ax1.set_title("KNN: %s" % outliers(knn_resid), size = 18, x = .485, ha = 'center')

ax2 = sns.kdeplot(rf_resid, ax=ax[0, 1])
ax2 = sns.kdeplot(norm, ax=ax[0, 1])
ax2.set_title("RF: %s" % outliers(rf_resid), size = 18, x = .485, ha = 'center')

ax3 = sns.kdeplot(gbr_resid, ax=ax[1, 0])
ax3 = sns.kdeplot(norm, ax=ax[1, 0])
ax3.set_title("GBR: %s" % outliers(gbr_resid), size = 18, x = .485, ha = 'center')

ax4 = sns.kdeplot(xgb_resid, ax=ax[1, 1])
ax4 = sns.kdeplot(norm, ax=ax[1, 1])
ax4.set_title("XGB: %s" % outliers(xgb_resid), size = 18, x = .485, ha = 'center')

resid_fig.suptitle("Standardized Residuals \n(normal dist. in red)", weight = 'bold', size = 18, y = 1.12)

resid_fig.text(x = 0, y = 0,
    s = '_________________________________________________________',
    fontsize = 14, color = 'grey', horizontalalignment='left', alpha = .3)

resid_fig.text(x = 0, y = -.06,
    s = 'https://dribbleanalytics.blog                     ',
    fontsize = 14, fontname = 'Rockwell', color = 'grey', horizontalalignment='left')

resid_fig.savefig('resid-fig-1.png', dpi = 400, bbox_inches = 'tight')

# Shapiro-Wilk test

In [ ]:
# perform shapiro-wilk test for normality of residuals

print(stats.shapiro(knn_resid))
print(stats.shapiro(rf_resid))
print(stats.shapiro(gbr_resid))
print(stats.shapiro(xgb_resid))

# Q-Q plot

In [ ]:
# q-q plot compared to a normal distribution

plt.rcParams.update(plt.rcParamsDefault)
plt.style.use('fivethirtyeight')

qqplot, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharex = True, sharey = True)

stats.probplot(knn_resid, dist="norm", plot=ax1)
ax1.set_xlabel("")
ax1.set_ylabel("")
ax1.set_title("KNN")

stats.probplot(rf_resid, dist="norm", plot=ax2)
ax2.set_xlabel("")
ax2.set_ylabel("")
ax2.set_title("RF")

stats.probplot(gbr_resid, dist="norm", plot=ax3)
ax3.set_xlabel("")
ax3.set_ylabel("")
ax3.set_title("GBR")

stats.probplot(xgb_resid, dist = "norm", plot = ax4)
ax4.set_xlabel("")
ax4.set_ylabel("")
ax4.set_title("XGB")

qqplot.text(0.5, -0.02, 'Theoretical Quantiles', ha='center', va='center', size = 18)
qqplot.text(0.01, 0.5, 'Ordered Values', ha='center', va='center', rotation='vertical', size = 18)

qqplot.text(x = 0, y = -0.05,
    s = '_______________________________________________________________',
    fontsize = 14, color = 'grey', horizontalalignment='left', alpha = .3)

qqplot.text(x = 0, y = -.1,
    s = 'https://dribbleanalytics.blog                     ',
    fontsize = 14, fontname = 'Rockwell', color = 'grey', horizontalalignment='left')

qqplot.savefig('qqplot.png', dpi = 400, bbox_inches = 'tight')

# Durbin-Watson test

In [ ]:
# perform durbin-watson test for autocorrelation

print(stattools.durbin_watson(knn_resid))
print(stattools.durbin_watson(rf_resid))
print(stattools.durbin_watson(gbr_resid))
print(stattools.durbin_watson(xgb_resid))

# Make predictions

In [ ]:
# function to create predictions from each model

def make_pred(model_list, df_pred):
    prob_list = []
    
    for i in model_list:
        prob_list.append(i.predict(df_pred))
        
    return prob_list

In [ ]:
# clean predictions and save to csv

prob_list = make_pred([knn, rf, gbr, xgb], df_2018[features])

pred_vals = pd.DataFrame(data = np.transpose(prob_list), columns = ['knn', 'rf', 'gbr', 'xgb'])

pred_vals['avg'] = (pred_vals['knn'] + pred_vals['rf'] + pred_vals['gbr'] + pred_vals['xgb']) / 4

In [ ]:
df_salary_pred = df_2018.join(pred_vals)
df_salary_pred['diff'] = df_salary_pred['avg'] - df_salary_pred['percent_of_cap']

df_salary_pred.to_csv('salary-pred.csv', index = False)

# An example on the effect of age

In [ ]:
# show how Luka Doncic's predicted salary changes if we suppose he's 27 years old instead of 19

doncic = df_2018[df_2018['player'] == 'Luka Doncic'].reset_index(drop = True)

doncic.loc[1] = doncic.loc[0]
doncic.loc[1, 'age'] = 27

In [ ]:
doncic

In [ ]:
prob_list = make_pred([knn, rf, gbr, xgb], doncic[features])

pred_vals = pd.DataFrame(data = np.transpose(prob_list), columns = ['knn', 'rf', 'gbr', 'xgb'])

pred_vals['avg'] = (pred_vals['knn'] + pred_vals['rf'] + pred_vals['gbr'] + pred_vals['xgb']) / 4

In [ ]:
pred_vals

# Evaluate predictions

In [ ]:
df_salary_pred = pd.read_csv('salary-pred.csv')

In [ ]:
# compare distribution of predicted salary to distribution of actual salary

plt.style.use('fivethirtyeight')

pred_obs_dens, ax = plt.subplots()

ax = sns.kdeplot(df_salary_pred['percent_of_cap'], ax = ax, shade = True, label = 'Observed salary')
ax = sns.kdeplot(df_salary_pred['avg'], ax = ax, shade = True, label = 'Predicted salary')

ax.set_xlabel("% of cap")
ax.set_ylabel("Density")

pred_obs_dens.suptitle("Observed and predicted salary", weight = 'bold', size = 18)

pred_obs_dens.text(x = -0.02, y = -0.08,
    s = '_____________________________________________________________',
    fontsize = 14, color = 'grey', horizontalalignment='left', alpha = .3)

pred_obs_dens.text(x = -0.02, y = -.14,
    s = 'https://dribbleanalytics.blog                     ',
    fontsize = 14, fontname = 'Rockwell', color = 'grey', horizontalalignment='left')

pred_obs_dens.savefig('pred_obs_dens.png', dpi = 400, bbox_inches = 'tight')

In [ ]:
# function to make bar graph from predictions

def make_plot(df_rows, color, title, neg_bool, height, file_name):
    
    plt.style.use('fivethirtyeight')
    
    fig, ax = plt.subplots()
    
    y = df_rows['diff']
    labels = df_rows['player']
    x = range(len(y))
    
    ax.bar(x, y, width = .7, edgecolor = 'white', color = color, linewidth = 4, label = 'Predicted')

    rects = ax.patches
    
    if neg_bool == True:
        for rect, label in zip(rects, labels):
            ax.text(rect.get_x() + rect.get_width() / 1.75, height, label,
            ha='center', va='bottom', rotation = 'vertical', color = 'black')
    else:
        for rect, label in zip(rects, labels):
            ax.text(rect.get_x() + rect.get_width() / 2.5, -height, label,
            ha='center', va='top', rotation = -90, color = 'black')

    fig.suptitle("%s" % title, weight = 'bold', size = 18)
    ax.xaxis.set_visible(False)
    
    ax.set_ylabel("Expected - actual % of cap")

    fig.text(x = -0.08, y = 0.03,
        s = '______________________________________________________________',
        fontsize = 14, color = 'grey', horizontalalignment='left', alpha = .3)

    fig.text(x = -0.08, y = -.03,
        s = 'https://dribbleanalytics.blog                     ',
        fontsize = 14, fontname = 'Rockwell', color = 'grey', horizontalalignment='left')

    fig.savefig('%s.png' % file_name, dpi = 400, bbox_inches = 'tight')

In [ ]:
best_val = df_salary_pred.sort_values(by = 'diff', ascending = False).reset_index(drop = True).iloc[0:10]

make_plot(best_val, 'springgreen', "Best bargain contracts", True, 0.005, 'best-val')

In [ ]:
worst_val = df_salary_pred.sort_values(by = 'diff', ascending = True).reset_index(drop = True).iloc[0:10]

make_plot(worst_val, 'lightcoral', "Worst value contracts", False, 0.005, 'worst-val')

In [ ]:
# groupby into team sum of difference and prepare data to run for make plot function

team_cap = df_salary_pred.groupby('tm')['diff'].sum().sort_values()
team_cap = pd.DataFrame(team_cap).reset_index().rename(columns = {'tm': 'player'})

In [ ]:
best_team = team_cap.sort_values(by = 'diff', ascending = False).iloc[:10]

make_plot(best_team, 'springgreen', "Best bargaining teams", True, 0.01, 'best-val-teams')

In [ ]:
worst_team = team_cap.sort_values(by = 'diff', ascending = True).iloc[:10]

make_plot(worst_team, 'lightcoral', "Worst bargaining teams", False, 0.01, 'worst-val-teams')

# SHAP values

In [ ]:
# prepare shap plots

shap.initjs()
plt.rcParams.update(plt.rcParamsDefault)
plt.style.use('fivethirtyeight')

In [ ]:
# function to create a plot of shap values given the values and model name

def plot_shap(shap_values, model_name):
    
    fig, ax = plt.subplots()
    
    ax = shap.summary_plot(shap_values, xtrain, show = False)
    fig.suptitle("%s Shapley values" % model_name.upper(), weight = 'bold', size = 18)
    
    fig.text(x = 0, y = -0.03,
        s = '________________________________________________________________________',
        fontsize = 14, color = 'grey', horizontalalignment='left', alpha = .3)

    fig.text(x = 0, y = -.075,
        s = 'https://dribbleanalytics.blog                     ',
        fontsize = 12, fontname = 'Rockwell', color = 'grey', horizontalalignment='left')

    fig.savefig('%s_shap.png' % model_name, dpi = 400, bbox_inches = 'tight')

In [ ]:
knn_k = shap.kmeans(xtrain, 5)

explainer = shap.KernelExplainer(knn.predict, knn_k)
shap_values = explainer.shap_values(xtrain)

plot_shap(shap_values, 'knn')

In [ ]:
explainer = shap.TreeExplainer(rf)
shap_values = explainer.shap_values(xtrain)

plot_shap(shap_values, 'rf')

In [ ]:
explainer = shap.TreeExplainer(gbr)
shap_values = explainer.shap_values(xtrain)

plot_shap(shap_values, 'gbr')

In [ ]:
explainer = shap.TreeExplainer(xgb)
shap_values = explainer.shap_values(xtrain)

plot_shap(shap_values, 'xgb')